In [1]:
# CNN, exit polls
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import os
import numpy as np
import pandas as pd
import pandas as pd
import json
import traceback

import sys
print(sys.version)

3.4.5 |Anaconda 4.1.1 (64-bit)| (default, Jul  5 2016, 14:53:07) [MSC v.1600 64 bit (AMD64)]


In [26]:
# get lists of paths to use with requests
paths = []
d = webdriver.Chrome()
d.implicitly_wait(10)
d.get('http://www.cnn.com/election/results/exit-polls')
xpath_state = '//*[@id="mount"]/div/main/div/section[1]/div/div[1]/section/div/div[1]/div[1]/select'
xpath_type = '//*[@id="mount"]/div/main/div/section[1]/div/div[1]/section/div/div[1]/div[2]/select'
el_state = d.find_element_by_xpath(xpath_state)
options_state = [x.get_attribute('value') for x in el_state.find_elements_by_tag_name("option")]
texts_state = [x.get_attribute('text') for x in el_state.find_elements_by_tag_name("option")]
for state, state_text in zip(options_state, texts_state):
    s_state = Select(d.find_element_by_xpath(xpath_state))
    s_state.select_by_value(state)
    time.sleep(5)
    el_type = d.find_element_by_xpath(xpath_type)
    options_type = [x.get_attribute('text') for x in el_type.find_elements_by_tag_name("option")]
    for type_ in options_type:
        paths.append([state_text, type_])
d.quit()

In [28]:
with open('cnn_paths', 'w+') as f:
    f.write(json.dumps(paths))

In [40]:
for p1, p2 in paths:
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get('http://www.cnn.com/election/results/exit-polls/{}/{}'.format(p1.lower(), p2.lower()))
    time.sleep(10)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    outerhtml = driver.execute_script("return document.documentElement.outerHTML")
    fn = p1+'_'+p2+'.html'
    with open(fn, 'w+', encoding='utf-8') as f:
        f.write(outerhtml)

In [103]:
results = []
for state, section in paths:
    with open(state+'_'+section+'.html', 'r') as f:
        html = f.read()
    soup = BeautifulSoup(html, 'lxml')
    frames = soup.find_all('div', attrs={'class': 'exit-poll-table'})
    for i, frame in enumerate(frames):
        question = frame.find_all('h3', attrs={'class': 'exit-poll__question'})[0].text
        respondents = frame.find_all('div', attrs={'class': 'exit-poll-table__metadata'})[0].text
        respondents = re.search('(\d+) respondents', respondents).group(1)
        respondents = int(respondents)
        divisions = []
        for th in frame.find_all('th', attrs={'class': "exit-poll__cell--header"}):
            if 'exit-poll__cell--answer' not in th.attrs['class']:
                divisions.append(th.attrs['data-lname'])
        tbody = frame.find_all('tbody')[0]
        for tr in tbody.find_all('tr'):
            answer = tr.find_all('td', attrs={'class': 'exit-poll__cell exit-poll__cell--answer'})[0].contents[1]
            pct_answer = tr.find_all('span', attrs={'class': 'exit-poll__answer-pct-value'})[0].contents[0]
            if pct_answer == 'N/A':
                pct_answer = None
            else:
                pct_answer = float(pct_answer.replace('%', ''))
            n = 0
            for cell in tr.find_all('td', attrs={'class': 'exit-poll__cell'}):
                if 'exit-poll__cell--answer' not in cell.attrs['class']:
                    pct = cell.contents[0]
                    if pct == 'N/A':
                        pct = None
                    else:
                        pct = float(pct.replace('%', ''))
                    results.append({'state': state, 'election': section, 'question_number': i, 'question': question,
                      'answer': answer, 'answer_pct': pct_answer, 'vote_group': divisions[n], 'cell_pct': pct})
                    n += 1
            assert n == len(divisions)


In [104]:
import pandas as pd
df = pd.DataFrame(results)

In [106]:
df.to_csv('USA_election_2016_exit_polls_cnn.csv', index=False)